In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import time
%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


# split train set in 0.8/0.2 train/validation sets
train_size = int(0.8 * len(trainset))
valid_size = len(trainset) - train_size

trainset, validset = torch.utils.data.random_split(trainset, [train_size, valid_size])

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

input_dim = 3 * 32 * 32
output_dim = 10
batch_size = 64

print("Train set size: {}, Valid set size: {}, Test set size: {}".format(
        len(trainset), len(validset), len(testset)
        ))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

def test(model, dataloader, batch_size=32):
    model.to(device)
    model.eval()
    
    total = 0
    correct = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            # send to device
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return 100 * correct / total;

Files already downloaded and verified
Files already downloaded and verified
Train set size: 40000, Valid set size: 10000, Test set size: 10000


### Simple CNN model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1, padding=1), # N x 16 x 32 x 32
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, stride=2), # N x 16 x 16 x 16
            nn.Conv2d(16, 32, 3, stride=1, padding=1), # N x 32 x 16 x 16
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, stride=2), # N x 32 x 8 x 8
        )
        
        self.fc = nn.Sequential(
            nn.Linear(32*8*8, 100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(100, 10)
        )
    
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

### With DataLoader

In [4]:
model = Net()
model.to(device)
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=0)

tic = time.time()
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: [{}\{}], loss: {:.4f}, iters: {}".format(epoch + 1, epochs, loss.item(), i))

toc = time.time()
print("Training time: {:.4f}".format(toc - tic))

train_acc = test(model, trainloader, batch_size=batch_size)
print("Train acc: ", train_acc)

Epoch: [1\5], loss: 1.8001, iters: 624
Epoch: [2\5], loss: 1.6919, iters: 624
Epoch: [3\5], loss: 1.4898, iters: 624
Epoch: [4\5], loss: 1.3814, iters: 624
Epoch: [5\5], loss: 1.4773, iters: 624
Training time: 38.8003
Train acc:  50.99


### Without DataLoader

In [5]:
train_x = []
train_y = []
batches_x = []
batches_y = []
i = 0
for x, y in trainset:
    i += 1
    train_x.append(x)
    train_y.append(y)
    
    if i % batch_size == 0:
        train_x = torch.stack(train_x).view(batch_size, 3, 32, 32)
        train_y = torch.tensor(train_y)
        batches_x.append(train_x)
        batches_y.append(train_y)
        train_x = []
        train_y = []

In [6]:
model = Net()
model.to(device)
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=0)

tic = time.time()
epochs = 5
for epoch in range(epochs):
    epoch_loss = 0
    for i, (images, labels) in enumerate(zip(batches_x, batches_y)):
        images, labels = images.to(device), labels.to(device)
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print("Epoch: [{}\{}], loss: {:.4f}, iters: {}".format(epoch + 1, epochs, loss.item(), i))

toc = time.time()
print("Training time: {:.4f}".format(toc - tic))

train_acc = test(model, trainloader, batch_size=batch_size)
print("Train acc: ", train_acc)

Epoch: [1\5], loss: 1.8431, iters: 624
Epoch: [2\5], loss: 1.6705, iters: 624
Epoch: [3\5], loss: 1.5499, iters: 624
Epoch: [4\5], loss: 1.4520, iters: 624
Epoch: [5\5], loss: 1.3730, iters: 624
Training time: 19.9167
Train acc:  51.905
